#  Introduction

This notebook presents the methodology for performing a volumetric balance in the leaching area of a mining operation. The leaching process involves monitoring and managing fluid flows between heaps and ponds in the extraction of iodine from caliche ore.

The primary goal of this analysis is to use data collected from field sensors to calculate inflows, outflows, and accumulation rates in the system, in order to obtain the volumetric balance. 

By identifying missing or incomplete data, this notebook will be the input to perform data reconciliation and determine the sensors to be installed in the process.

A flowsheet of the leaching process is shown in the portfolio pdf document.

# Libraries

In [1]:
import pandas as pd
import datetime

# Data loading

In [2]:
# In this section, we will load the raw datasets necessary for performing the volumetric balance calculations. 

# DB_LEACHING excel is a manually created information document:
# The "heaps" sheet details which sector each pile belongs to.
# The "Nodes" sheet details which sector the nodes belong to within the mine and whether there are piles and/or ponds.
# The "flows" sheet indicates the description of the input and output flows of each node.

df_heaps=pd.read_excel("DB_leaching.xlsx",sheet_name='heaps',dtype=str)
df_nodes=pd.read_excel("DB_leaching.xlsx",sheet_name='nodes')
df_flows=pd.read_excel("DB_leaching.xlsx",sheet_name='flows')
#--------------------------------------------------------------------------------------------------------------------
 
# Query_db excel was obtained from a SQL query to the database that stores the information on the irrigation and drainage of 
# the different heaps, the consulted data base has daily information, the SQL query used is in the github repository.

df_irrigation_drainage=pd.read_excel('query_db.xlsx')
columns_irrigation_drainage=['heap','timestamp','Type','irrigation','type_irrigation','real_drainage','stage']

# We only keep the columns that interest us, we transform the irrigation and drainage columns of heaps to leave everything 
# in cubic meters and we cross the information with df_heaps to add the sector information.

df_irrigation_drainage=df_irrigation_drainage[columns_irrigation_drainage]
df_irrigation_drainage['timestamp']=pd.to_datetime(df_irrigation_drainage.loc[:,'timestamp'], format='%Y-%m-%d')
df_irrigation_drainage['irrigation']=(df_irrigation_drainage.loc[:,'irrigation'])/1000000
df_irrigation_drainage['real_drainage']=(df_irrigation_drainage.loc[:,'real_drainage'])/1000000
df_irrigation_drainage['heap']=df_irrigation_drainage.loc[:,'heap'].astype(str)
df_irrigation_drainage=pd.merge(df_irrigation_drainage, df_heaps, on=['heap'], how='left') 
df_irrigation_drainage.dropna(inplace=True)
#--------------------------------------------------------------------------------------------------------------------
                                                                              
# Data is downloaded from the database to obtain the flows of the different solutions: water, intermediate solution (SI),
# acid weak water (AFA) and brine, for irrigation and drainage.
                                                                              
df_sol=pd.read_excel("solutions_flow.xlsx",sheet_name="solutions")
df_sol.columns=df_sol.columns.astype('str')
df_sol=df_sol.fillna(0)
df_sol=df_sol[1:-1]
df_sol.iloc[:,2:]=df_sol.iloc[:,2:]*24

df_water=pd.read_excel("solutions_flow.xlsx",sheet_name="water")
df_water.columns=df_water.columns.astype('str')
df_water=df_water.fillna(0)
df_water.columns=['Month', 'Day']+df_water.iloc[0,2:].tolist()
df_water=df_water[2:-1]

df_measurements=pd.read_excel("solutions_flow.xlsx",sheet_name="measurements")
df_measurements=df_measurements.fillna(0)
#--------------------------------------------------------------------------------------------------------------------
                                                                              
# df_evap is an excel that has the average monthly evaporation data in the sector for both water and solutions.

df_evap=pd.read_excel('pond_evap.xlsx')                                                                             
#--------------------------------------------------------------------------------------------------------------------
                                                                              
# Pond dimensions is an excel that has the area of the ponds of each sector and of each solution, this will be used to 
# calculate the losses due to evaporation.

df_pond_dim=pd.read_excel('pond_dimensions.xlsx')

# Data Cleaning and Processing

In [3]:
# The time interval for performing the balance is chosen
df_sol=df_sol[df_sol['Day']==14] 
df_water=df_water[df_water['Day']==14]
df_evap=df_evap[df_evap['Month']=='March']
df_irrigation_drainage=df_irrigation_drainage[df_irrigation_drainage['timestamp']=='2022-03-14']

# A Dictionary of {Sector:[Heap]} is generated
heap_dict={sector:df_irrigation_drainage[df_irrigation_drainage['Sector']==sector]['heap'].tolist() 
            for sector in df_irrigation_drainage['Sector'].unique()}

# Balances to define unmeasured flows

In [4]:
# Review of which flows do not exist in the "SIPRO" database, and must be obtained through balances
df_flows[df_flows['SIPRO description']=='It does not exist'].loc[:,['Node','pond','Flow','Comment']]

,Node,pond,Flow,Comment
16,SC-7,brine,out,ok
17,SC-6,brine,in,ok
45,CO4,mixture,out,ok
63,CO2,si-plant,in,Parte de dren CO2A
65,CO2,water,in,ok
68,CO2,water,out,ok
72,CO2,afa,in,ok
74,Intermediate pond,si,in,Parte de dren CO2A
77,Intermediate pond,si,in,ok
78,Torcaza,water,in,"Mediciones, usar ""water Llamara a CO6"" ojo es ..."


Now each sector will be reviewed

## Sector: SC 7 - SC-6

In [5]:
# The pond level will be considered constant
# The result is added to the dataframe "df_sol"
df_sol['Brine SC-7 a SC-6']=df_irrigation_drainage[(df_irrigation_drainage['stage']=='Head') & (df_irrigation_drainage['Sector']=='SC-2')]['real_drainage'].sum()-(
    df_pond_dim[df_pond_dim['Sector']=='SC-7']['Area(m2)'].item())*(df_evap['Solution (Prec)'].item())/1000

In [6]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (
    (df_flows['Node']=='SC-7')|(df_flows['Node']=='SC-6')),['SIPRO description'] ]='Brine SC-7 a SC-6'

## Sector: CO4

In [7]:
# The pond level will be considered constant
# The result is added to the dataframe "df_sol"

df_sol['AFA CO4 a BC']=df_water['Mezcla CO4'].item()+df_sol['AFA a CO4'].item()-df_measurements[df_measurements['Measurement']=='AFA a CO6']['Total'].item()-(
    df_pond_dim[(df_pond_dim['Sector']=='CO4') & (df_pond_dim['Pond']=='mixture')]['Area(m2)'].item())*(
    df_evap['Solution (Prec)'].item())/1000

In [8]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='CO4'),
              ['SIPRO description'] ]='AFA CO4 a BC'

## Sector: CO2

### si-plant

In [9]:
# The pond level will be considered constant
# The result is added to the dataframe "df_sol"
df_sol['SI intermedia a CO2']=df_sol['Brine a Iris'].item()+df_pond_dim[(df_pond_dim['Sector']=='CO2') & 
(df_pond_dim['Pond']=='si-plant')]['Area(m2)'].item()*(df_evap['Solution (Prec)'].item())/1000

In [10]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='CO2')& (df_flows['pond']=='si-plant'),
              ['SIPRO description'] ]='SI intermedia a CO2'

### Intermediate pond

In [11]:
# Intermediate solution (SI)
# The result is added to the dataframe "df_sol"
df_sol['SI a Poza intermedia drenaje']=(df_irrigation_drainage[(df_irrigation_drainage['Sector']=='CO2A') & 
                                                               (df_irrigation_drainage['stage']=='Tail') &
(df_irrigation_drainage['heap']!='540')]['real_drainage'].sum()-df_sol['SI intermedia a CO2'].item())

In [12]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='Intermediate pond')& 
              (df_flows['Solution']=='si'),['SIPRO description'] ]='SI a Poza intermedia drenaje'

In [13]:
# Water
# The pond level will be considered constant
# The result is added to the dataframe "df_sol"

df_water['Agua a Poza intermedia']=df_sol['Intermedia a CO2A'].item()-df_sol['SI CO2 Intermedia'].item() + df_pond_dim[
    (df_pond_dim['Sector']=='Intermediate pond') & (df_pond_dim['Pond']=='si')]['Area(m2)'].item()*(
    df_evap['Solution (Prec)'].item())/1000 -df_sol['SI a Poza intermedia drenaje'].item()

In [14]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='Intermediate pond')& 
              (df_flows['Solution']=='water'),['SIPRO description'] ]='Agua a Poza intermedia'

In [15]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='CO2')& 
              (df_flows['Solution']=='water') & (df_flows['Flow']=='out') ,
              ['SIPRO description']]='Agua a Poza intermedia'

### Water pond

In [16]:
# Water
# The pond level will be considered constant
# The result is added to the dataframe "df_sol"

df_water['Agua a CO2']=df_water['SI CO2A'].item() + df_water['Mezcla CO2A'].item() + df_water['Agua a Poza intermedia'].item() + df_pond_dim[
    (df_pond_dim['Sector']=='CO2') & (df_pond_dim['Pond']=='water')]['Area(m2)'].item()*(
    df_evap['Water'].item())/1000 

In [17]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='CO2')& 
              (df_flows['Solution']=='water'),['SIPRO description'] ]='Agua a CO2'

### AFA pond

In [18]:
# Afa
# The pond level will be considered constant
# The result is added to the dataframe "df_sol"


df_sol['AFA Iris a CO2']=df_sol['AFA CO2 a CO2A'].item() + df_pond_dim[
    (df_pond_dim['Sector']=='CO2') & (df_pond_dim['Pond']=='afa')]['Area(m2)'].item()*(
    df_evap['Solution (Prec)'].item())/1000 - df_measurements[df_measurements['Measurement']=='AFA CO1 a CO2']['Total'].item()

In [19]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='CO2')& 
              (df_flows['Solution']=='afa'),['SIPRO description'] ]='AFA Iris a CO2'



In [20]:
# Some flows measurements are considered fixed in data reconciliation, because they are regulated by the authority

df_flows.loc[(df_flows['SIPRO description']=='AFA Iris a CO2'),['Fixed value']]=1

## Sector: Torcaza

In [21]:
# Water pond
# Water flow balance

df_water['Agua a Torcaza']= df_measurements[df_measurements['Measurement']=='Agua Llamara a CO6']['Total'].item() - df_water['Agua MZ CO6'].item() 

In [22]:
df_flows.loc[(df_flows['SIPRO description']=='It does not exist')& (df_flows['Node']=='Torcaza'),
              ['SIPRO description'] ]='Agua a Torcaza'

## Final check

In [23]:
df_flows[df_flows['SIPRO description']=='It does not exist'].loc[:,['Node','pond','Flow','Comment']]

,Node,pond,Flow,Comment


Now we have all the flows with information

# Functions

Three functions are defined to perform the balance in each pond, the functions are to determine the inputs, outputs and evaporation losses.

In [24]:
def pond_input(sectors):
    """ Provide a dictionary with the sum of the M3 of input to each pond, according to the chosen sector"""
    
    global dict_in_pond_sectors
    dict_in_pond_sectors={}
    for sector in sectors:
        global dict_in_pond
        dict_in_pond={}
    
        # It goes through each pond in the sector
        for pond in df_flows[df_flows['Node']==sector]['pond'].unique():
    
            # Descriptions of SIPRO database input flows are saved
            descriptions=df_flows[(df_flows['Node']==sector) & (df_flows['pond']==pond) & 
                  (df_flows['Flow']=='in')]['SIPRO description']
        
            # The following situations are considered:
                # SI: water, drainage and solutions
                # Mixture: water and solutions
                # Afa: Afa
                # Water: water
                # Brine: solutions and drainage
    
            if ((pond=='si') or (pond=='si-plant') or (pond=='si-irrigation')) :
                si_in=0
                for desc_sipro in descriptions:
                    
                    if desc_sipro in df_water.columns:
                        si_in+=df_water[desc_sipro].item()

                    elif desc_sipro in df_sol.columns:
                            si_in+=(df_sol[desc_sipro]).item()

                    elif desc_sipro in list(df_measurements['Measurement']):
                        si_in+=df_measurements[df_measurements['Measurement']==desc_sipro]['Total'].item()

                    elif desc_sipro=='drainage':
                        # Special case of the 540 heap from Cancri
                        if df_flows[(df_flows['Node']==sector) & (df_flows['pond']==pond) & 
                   (df_flows['Flow']=='in') & (df_flows['SIPRO description']=='drainage')]['heap drainage'].item()=='Pila 540':
                            si_in=df_irrigation_drainage[df_irrigation_drainage['heap']=='540']['real_drainage'].sum()
                        
                        else:
                            input_drainage=df_flows[(df_flows['Node']==sector) & (df_flows['pond']==pond) & 
                       (df_flows['Flow']=='in') & (df_flows['SIPRO description']=='drainage')]['heap drainage'].item()
                            list_heap_drain=[heap for heap in heap_dict[input_drainage] if heap in df_irrigation_drainage
                                      [df_irrigation_drainage['stage']=='Tail']['heap'].unique()]
                            si_in+=df_irrigation_drainage[df_irrigation_drainage['heap'].isin(list_drain_pond)]['real_drainage'].sum()

                dict_in_pond[pond]=si_in
                
                
            if pond=='mixture':
                mixture_in=0
                for desc_sipro in descriptions:
                    
                    if desc_sipro in df_water.columns:
                        mixture_in+=df_water[desc_sipro].item()

                    elif desc_sipro in df_sol.columns:
                        mixture_in+=(df_sol[desc_sipro]).item()
                        
                    elif desc_sipro in list(df_measurements['Measurement']):
                        mixture_in+=df_measurements[df_measurements['Measurement']==desc_sipro]['Total'].item()
 
                dict_in_pond[pond]=mixture_in
        
            if pond=='afa':
                afa_in=0
                for desc_sipro in descriptions:
                
                    if desc_sipro in df_sol.columns:
                        afa_in+=(df_sol[desc_sipro]).item()
                        
                    elif desc_sipro in list(df_measurements['Measurement']):
                        afa_in+=df_measurements[df_measurements['Measurement']==desc_sipro]['Total'].item()
                    
                dict_in_pond[pond]=afa_in            
            
            if pond=='water':
                water_in=0
                for desc_sipro in descriptions:
                    
                    if desc_sipro in df_water.columns:
                        water_in+=df_water[desc_sipro].item()
                        
                    elif desc_sipro in list(df_measurements['Measurement']):
                        water_in+=df_measurements[df_measurements['Measurement']==desc_sipro]['Total'].item()
                dict_in_pond[pond]=water_in
                
            if ((pond=='brine') or (pond=='brine-torcaza') or (pond=='brine-cop 2')):
                brine_in=0
                for desc_sipro in descriptions:
                    
                    if desc_sipro in df_sol.columns:
                        brine_in+=(df_sol[desc_sipro]).item()
                
                    elif desc_sipro in list(df_measurements['Measurement']):
                        brine_in+=df_measurements[df_measurements['Measurement']==desc_sipro]['Total'].item()
                
                    elif desc_sipro=='drainage':
                        drainage_in=df_flows[(df_flows['Node']==sector) & (df_flows['pond']==pond) & 
                   (df_flows['Flow']=='in') & (df_flows['SIPRO description']=='drainage')]['heap drainage'].item()
                        list_drain_pond=[heap for heap in heap_dict[drainage_in] if heap in df_irrigation_drainage
                                         [df_irrigation_drainage['stage']=='Head']['heap'].unique()]
                        brine_in+=df_irrigation_drainage[df_irrigation_drainage['heap'].isin(list_drain_pond)]['real_drainage'].sum()
                
                dict_in_pond[pond]=brine_in
                
        dict_in_pond_sectors[sector]=dict_in_pond
        
    return dict_in_pond_sectors

In [25]:
def evap_pond(sectors):
    """Provides a dictionary with evaporation loss in cubic meters (M3) according to dimensions and evaporation rate for each pond"""
    
    global dict_evap_sector
    dict_evap_sector={}
    for sector in sectors:     
        global dict_evap_pond
        dict_evap_pond={}
        
        # It goes through each pond in the sector
        for pond in df_flows[df_flows['Node']==sector]['pond'].unique():
        
            # The following situations are considered:
                # Water: Water evaporation rate is used
                # Other: Solution evaporation rate is used (Prod)
    
            if pond=='water': 
                dict_evap_pond[pond]=(((df_pond_dim[(df_pond_dim['Sector']==sector) & (df_pond_dim['Pond']==pond)]['Area(m2)']).item())
                              *(df_evap['Water'].item())/1000)
    
            else:
                dict_evap_pond[pond]=(((df_pond_dim[(df_pond_dim['Sector']==sector) & (df_pond_dim['Pond']==pond)]['Area(m2)']).item())
                             *(df_evap['Solution (Prec)'].item())/1000)

                
        dict_evap_sector[sector]=dict_evap_pond
    return dict_evap_sector

In [26]:
def pond_output(sectors):
    """ Provides a dictionary with the sum of the cubic meters (M3) output from each pond, according to the chosen sector"""
    
    global dict_out_pond_sectors
    dict_out_pond_sectors={}
    for sector in sectors:
        dict_out_pond={}
        
        # It goes through each pond in the sector
        for pond in df_flows[df_flows['Node']==sector]['pond'].unique():
    
            # It is reviewed whether the output is a transfer of solutions
            if'out' in df_flows[(df_flows['Node']==sector) & (df_flows['pond']==pond)]['Flow'].unique():
            
                # Descriptions of SIPRO output streams are saved
                descriptions=df_flows[(df_flows['Node']==sector) & (df_flows['pond']==pond) & 
                               (df_flows['Flow']=='out')]['SIPRO description']
                out_pond=0
                
                for desc_sipro in  descriptions:
                    
                    if desc_sipro in df_sol.columns:
                        out_pond+=(df_sol[desc_sipro]).item()
                
                    if desc_sipro in df_water.columns:
                        out_pond+=df_water[desc_sipro].item()
                    
                    if desc_sipro in list(df_measurements['Measurement']):
                         out_pond+=df_measurements[df_measurements['Measurement']==desc_sipro]['Total'].item()
                            
                    if (desc_sipro=='irrigation') or (desc_sipro=='irrigation 540'):
                        if ((pond=='si') or (pond=='si-plant') or (pond=='si-irrigation')):# Assuming impregnation is only SI
                            out_pond=df_irrigation_drainage[(df_irrigation_drainage['Sector']==sector)
                             &((df_irrigation_drainage['type_irrigation']=='Impregnation')|(df_irrigation_drainage['type_irrigation']=='SI'))]['irrigation'].sum()
                            
                        if pond=='mixture':
                            out_pond=df_irrigation_drainage[(df_irrigation_drainage['Sector']==sector) &
                                           (df_irrigation_drainage['type_irrigation']=='Mixture') ]['irrigation'].sum()
                            
                        if pond=='water': 
                            out_pond=df_irrigation_drainage[(df_irrigation_drainage['Sector']==sector)
                            &((df_irrigation_drainage['type_irrigation']=='Water impregantion')|(df_irrigation_drainage['type_irrigation']=='Water'))]['irrigation'].sum()

                dict_out_pond[pond]=out_pond
   
        dict_out_pond_sectors[sector]=dict_out_pond
    return dict_out_pond_sectors

# Results

Now the function that integrates the 3 previous functions is defined, and gives the final result of the notebook

In [27]:
def balance(sectors):
    """ This function saves the entry and loss dictionaries for each pond"""
    pond_input(sectors)
    evap_pond(sectors)
    pond_output(sectors) 

In [28]:
# The "balance" function is applied and the balance is performed for each sector in steady state.
balance(df_nodes['Nodes'].unique())
steady_state_balance=(pd.DataFrame(dict_in_pond_sectors)-pd.DataFrame(dict_out_pond_sectors)-pd.DataFrame(dict_evap_sector)).fillna(0)
steady_state_balance

,SC-2,SC-7,SC-6,CO5,SC-3,Poza 30-31,CO4,COP-1,CO2,Poza intermedia,CO2A,Torcaza
afa,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,0.000000e+00,-3012.18908,-9.279000e+03,0.0,0.000,0.00000
brine,0.0,-7.602807e-13,-12179.83406,-9833.52446,0.0,0.0,1.669020e+02,0.00000,8.756432e+02,0.0,0.000,-48.07095
brine-cop 2,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,0.000000e+00,-11603.75353,0.000000e+00,0.0,0.000,0.00000
brine-torcaza,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,0.000000e+00,10558.52662,0.000000e+00,0.0,0.000,0.00000
mixture,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,-9.996312e+07,0.00000,0.000000e+00,0.0,-8457.758,0.00000
si,0.0,0.000000e+00,0.00000,-17624.77867,0.0,0.0,8.209902e+03,375.17743,0.000000e+00,0.0,-2181.758,0.00000
si-irrigation,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,0.000000e+00,0.00000,-3.957663e+02,0.0,0.000,0.00000
si-plant,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,0.000000e+00,0.00000,-4.298784e-13,0.0,0.000,0.00000
water,0.0,0.000000e+00,0.00000,0.00000,0.0,0.0,0.000000e+00,0.00000,9.260000e+03,0.0,0.000,0.00000


This result of the steady-state balance is the input for the data reconciliation carried out by the data science team. Once the reconciliation is finished, work is done on developing the final product, which is the visualization of the system in Power BI. This result is found in the GitHub repository.